In [1]:
import angr
import archinfo
arch = archinfo.ArchAArch64()

In [2]:
# Loading the elf fila and creating a project
path_to_binary = './tee.elf'
proj = angr.Project(path_to_binary, auto_load_libs=False)

In [3]:
cfg = proj.analyses.CFG()

WARNING  | 2024-05-06 17:14:48,928 | claripy.vsa.strided_interval | Tried to cast_low an interval to an interval shorter than its stride.
WARNING  | 2024-05-06 17:14:50,206 | claripy.vsa.strided_interval | Tried to cast_low an interval to an interval shorter than its stride.


In [ ]:
# Creating a initial state and simulation manager to perform symbolic execution on the particular function.
function_name = "syscall_open_ta_session"
function_address = proj.loader.find_symbol(function_name).rebased_addr
initial_state = proj.factory.blank_state(addr=function_address)
simgr = proj.factory.simulation_manager(initial_state, save_unconstrained=True)

# Defining functions to perform backtracking
def get_node_from_address(address):
    all_nodes = list(cfg.graph.nodes)
    for node in all_nodes:
        if node.addr == address:
            return node
    return None

def handle_spin_to_backtrack(LocalLoopSeer, succ_state):
    LocalLoopSeer.cut_succs.append(succ_state)

# Exploring with Breadth First Search (Default) and Local flag Seer to detect loops
simgr.use_technique(angr.exploration_techniques.LocalLoopSeer(bound=1, bound_reached=handle_spin_to_backtrack))

In [9]:
# Exploring with Depth First Search (Optional)
from angr.exploration_techniques import DFS
simgr.use_technique(DFS())

In [16]:
# Exploration Handler Functions
def get_function_name(project, current_pc):
    try:
        function = cfg.functions[current_pc]

    except (KeyError, AttributeError) as e:
        pass
    
def isValid(addr):
    try:
        proj.factory.block(int(addr, 16))
        return True
    except:
        return False

# Initializing Variables
count = 0
first_iter = True
branch_address_1 = None
branch_address_2 = None
branch_addresses = []
flag = True

while flag:
    for state in simgr.active:

        branch_address_2 = None
        branch_address_1 = None

        current_pc = state.regs.pc
        try:
            current_pc_int = current_pc.args[0]
        except Exception as e:
            pass

        current_block = proj.factory.block(state.addr)
        branch_insn = None
        for insn in current_block.capstone.insns:
            if insn == None:
                break
            branch_insn = insn
        if branch_insn == None:
            continue
        branch_address_1 = branch_insn.address+4
        branch_addresses.append(branch_address_1)
        branch_address_2 = branch_insn.op_str.split(",")[-1][1:]
        try:
            if isValid(branch_address_2):
                branch_address_2 = int(branch_address_2, 16)
                branch_addresses.append(branch_address_2)
        except ValueError:
            pass
        get_function_name(proj, current_pc_int)
    if flag == False:
        break
    print(branch_addresses)
    simgr.step()
    for b_addr in branch_addresses:
        add = True
        for state in simgr.active:
            if b_addr == state.addr:
                add = False
        if add:
            count = count + 1
            state = proj.factory.blank_state(addr=b_addr) # inherit parent properties here
            simgr.active.append(state)
        else:
            branch_addresses.remove(b_addr)

    if not first_iter:
        while count > 0:
            pop_addr = branch_addresses.pop(0)
            for state in simgr.active:
                if state.addr == pop_addr:
                    simgr.active.remove(state)
            count = count - 1
    first_iter = False

WARNING  | 2024-05-05 18:53:30,242 | angr.storage.memory_mixins.default_filler_mixin | Filling register x29 with 8 unconstrained bytes referenced from 0xe12a40c (syscall_open_ta_session+0x0 in tee.elf (0xe12a40c))
WARNING  | 2024-05-05 18:53:30,243 | angr.storage.memory_mixins.default_filler_mixin | Filling register x30 with 8 unconstrained bytes referenced from 0xe12a40c (syscall_open_ta_session+0x0 in tee.elf (0xe12a40c))
WARNING  | 2024-05-05 18:53:30,244 | angr.storage.memory_mixins.default_filler_mixin | Filling register x21 with 8 unconstrained bytes referenced from 0xe12a414 (syscall_open_ta_session+0x8 in tee.elf (0xe12a414))
WARNING  | 2024-05-05 18:53:30,245 | angr.storage.memory_mixins.default_filler_mixin | Filling register x22 with 8 unconstrained bytes referenced from 0xe12a414 (syscall_open_ta_session+0x8 in tee.elf (0xe12a414))
WARNING  | 2024-05-05 18:53:30,246 | angr.storage.memory_mixins.default_filler_mixin | Filling register x2 with 8 unconstrained bytes referenced

Simgr.active -  [<SimState @ 0xe12a40c>]
Current Address: 0xe12a40c
Current block instructions:
stp x29, x30, [sp, #-0x90]!
mov x29, sp
stp x21, x22, [sp, #0x20]
mov x22, x2
adrp x2, #0xe184000
stp x19, x20, [sp, #0x10]
mov x19, x0
ldr x2, [x2, #0x2f8]
stp x23, x24, [sp, #0x30]
mov x24, x3
stp x25, x26, [sp, #0x40]
mov x23, x4
stp x27, x28, [sp, #0x50]
str x1, [sp, #0x68]
ldr x0, [x2]
str x0, [sp, #0x88]
mov x0, #0
bl #0xe117e44
Current Function: syscall_open_ta_session
-----------------------------
[236102740, 236027460]
Simgr.active -  [<SimState @ 0xe117e44>, <SimState @ 0xe12a454>]
Current Address: 0xe117e44
Current block instructions:
stp x29, x30, [sp, #-0x10]!
mov x29, sp
bl #0xe117e2c
Current Function: ts_get_current_session
-----------------------------
Simgr.active -  [<SimState @ 0xe117e44>, <SimState @ 0xe12a454>]
Current Address: 0xe12a454
Current block instructions:
mov x28, x0
ldr x0, [x0, #0x10]
bl #0xe129cf4
-----------------------------
[236102740, 236027472, 23602743

WARNING  | 2024-05-05 18:53:30,545 | angr.storage.memory_mixins.default_filler_mixin | Filling memory at 0xf000000000000010 with 8 unconstrained bytes referenced from 0xe11b1cc (is_user_ta_ctx+0x4 in tee.elf (0xe11b1cc))
WARNING  | 2024-05-05 18:53:30,642 | angr.storage.memory_mixins.default_filler_mixin | Filling register x20 with 8 unconstrained bytes referenced from 0xe15cdd8 (malloc+0x8 in tee.elf (0xe15cdd8))
WARNING  | 2024-05-05 18:53:30,643 | angr.storage.memory_mixins.default_filler_mixin | Filling register x21 with 8 unconstrained bytes referenced from 0xe15cde8 (malloc+0x18 in tee.elf (0xe15cde8))
WARNING  | 2024-05-05 18:53:30,646 | angr.storage.memory_mixins.default_filler_mixin | Filling register x20 with 8 unconstrained bytes referenced from 0xe15ebd4 (trace_printf+0x8 in tee.elf (0xe15ebd4))
WARNING  | 2024-05-05 18:53:30,647 | angr.storage.memory_mixins.default_filler_mixin | Filling register x21 with 8 unconstrained bytes referenced from 0xe15ebe4 (trace_printf+0x18 i

Simgr.active -  [<SimState @ 0xe11ab70>, <SimState @ 0xe11b1e0>, <SimState @ 0xe129d08>]
Current Address: 0xe11ab70
Current block instructions:
stp x29, x30, [sp, #-0x20]!
mov w0, #1
mov x29, sp
stp x19, x20, [sp, #0x10]
bl #0xe104a88
Current Function: thread_get_id_may_fail
-----------------------------
Simgr.active -  [<SimState @ 0xe11ab70>, <SimState @ 0xe11b1e0>, <SimState @ 0xe129d08>]
Current Address: 0xe11b1e0
Current block instructions:
ret 
-----------------------------
Simgr.active -  [<SimState @ 0xe11ab70>, <SimState @ 0xe11b1e0>, <SimState @ 0xe129d08>]
Current Address: 0xe129d08
Current block instructions:
tst w0, #0xff
b.ne #0xe129d34
-----------------------------
[236040684, 236040676, 236039044, 235948680, 236040676, 236100880, 236100916]
Simgr.active -  [<SimState @ 0xe104a88>, <SimState @ 0xe129d08>, <SimState @ 0xe129d34>, <SimState @ 0xe129d10>]
Current Address: 0xe104a88
Current block instructions:
stp x29, x30, [sp, #-0x20]!
and w1, w0, #7
mov x29, sp
str x19, [

WARNING  | 2024-05-05 18:53:30,774 | angr.storage.memory_mixins.default_filler_mixin | Filling register x20 with 8 unconstrained bytes referenced from 0xe116630 (__do_panic+0x8c in tee.elf (0xe116630))
WARNING  | 2024-05-05 18:53:30,803 | angr.storage.memory_mixins.default_filler_mixin | Filling register x20 with 8 unconstrained bytes referenced from 0xe1165d8 (__do_panic+0x34 in tee.elf (0xe1165d8))
WARNING  | 2024-05-05 18:53:30,863 | angr.storage.memory_mixins.default_filler_mixin | Filling register x29 with 8 unconstrained bytes referenced from 0xe104998 (select_vector_wa_spectre_bhb+0x0 in tee.elf (0xe104998))
WARNING  | 2024-05-05 18:53:30,864 | angr.storage.memory_mixins.default_filler_mixin | Filling register x30 with 8 unconstrained bytes referenced from 0xe104998 (select_vector_wa_spectre_bhb+0x0 in tee.elf (0xe104998))
WARNING  | 2024-05-05 18:53:30,865 | angr.storage.memory_mixins.default_filler_mixin | Filling register x19 with 8 unconstrained bytes referenced from 0xe1049

...

Simgr.active -  [<SimState @ 0xe112194>, <SimState @ 0xe15c424>, <SimState @ 0xe116644>, <SimState @ 0xe1038c8>, <SimState @ 0xe1039a4>, <SimState @ 0xe10398c>, <SimState @ 0xe11a954>, <SimState @ 0xe1038c0>, <SimState @ 0xe112180>, <SimState @ 0xe11a954>, <SimState @ 0xe1038c0>, <SimState @ 0xe15dcb0>, <SimState @ 0xe15ebb0>, <SimState @ 0xe15dcb0>, <SimState @ 0xe10498c>, <SimState @ 0xe10823c>, <SimState @ 0xe103970>, <SimState @ 0xe103984>, <SimState @ 0xe1165a4>, <SimState @ 0xe1165fc>, <SimState @ 0xe103968>, <SimState @ 0xe103920>, <SimState @ 0xe11aa20>, <SimState @ 0xe11abe8>, <SimState @ 0xe103930>, <SimState @ 0xe1039ac>, <SimState @ 0xe1121fc>, <SimState @ 0xe1121d8>, <SimState @ 0xe1082a0>]
Current Address: 0xe112194
Current block instructions:
sub sp, sp, #0x20
mov w7, w2
mov x6, x1
mov x5, x0
mov w2, #1
mov w1, #0
stp x29, x30, [sp, #0x10]
add x29, sp, #0x10
mov x0, #0
adrp x4, #0xe16e000
str x3, [sp]
add x4, x4, #0xba0
mov w3, #1
bl #0xe15ebcc
Current Function: _assert_

WARNING  | 2024-05-05 18:53:33,119 | angr.engines.successors | Exit state has over 256 possible solutions. Likely unconstrained; skipping. <BV64 reg_x30_1259_64{UNINITIALIZED}>
WARNING  | 2024-05-05 18:53:33,122 | angr.storage.memory_mixins.default_filler_mixin | Filling register x19 with 8 unconstrained bytes referenced from 0xe1165b0 (__do_panic+0xc in tee.elf (0xe1165b0))
WARNING  | 2024-05-05 18:53:33,122 | angr.storage.memory_mixins.default_filler_mixin | Filling register x20 with 8 unconstrained bytes referenced from 0xe1165b0 (__do_panic+0xc in tee.elf (0xe1165b0))
WARNING  | 2024-05-05 18:53:33,123 | angr.storage.memory_mixins.default_filler_mixin | Filling register x21 with 8 unconstrained bytes referenced from 0xe1165c0 (__do_panic+0x1c in tee.elf (0xe1165c0))
WARNING  | 2024-05-05 18:53:33,124 | angr.storage.memory_mixins.default_filler_mixin | Filling register x22 with 8 unconstrained bytes referenced from 0xe1165c0 (__do_panic+0x1c in tee.elf (0xe1165c0))
WARNING  | 2024-0

Simgr.active -  [<SimState @ 0xe15ebcc>, <SimState @ 0xe15c2ec>, <SimState @ 0xe116654>, <SimState @ 0xe1038d4>, <SimState @ 0xe10398c>, <SimState @ 0xe103994>, <SimState @ 0xe11a3a0>, <SimState @ 0xe1038c8>, <SimState @ 0xe104a44>, <SimState @ 0xe11a3a0>, <SimState @ 0xe1038c8>, <SimState @ 0xe1038c0>, <SimState @ 0xe15ec5c>, <SimState @ 0xe1038c0>, <SimState @ 0xe11a3ac>, <SimState @ 0xe15ebcc>, <SimState @ 0xe103970>, <SimState @ 0xe103984>, <SimState @ 0xe104a88>, <SimState @ 0xe1166f8>, <SimState @ 0xe103970>, <SimState @ 0xe103984>, <SimState @ 0xe1039a4>, <SimState @ 0xe103930>, <SimState @ 0xe104a88>, <SimState @ 0xe11ab70>, <SimState @ 0xe103930>, <SimState @ 0xe103948>, <SimState @ 0xe10a590>, <SimState @ 0xe1165a4>, <SimState @ 0xe138e94>, <SimState @ 0xe11a960>, <SimState @ 0xe103988>, <SimState @ 0xe11218c>, <SimState @ 0xe15dd1c>, <SimState @ 0xe104998>, <SimState @ 0xe108290>, <SimState @ 0xe1165d0>, <SimState @ 0xe11aa40>]
Current Address: 0xe15ebcc
Current block instru

WARNING  | 2024-05-05 18:53:33,987 | angr.storage.memory_mixins.default_filler_mixin | Filling memory at 0xffffffffffffff00 with 8 unconstrained bytes referenced from 0xe15ec5c (trace_printf+0x90 in tee.elf (0xe15ec5c))
WARNING  | 2024-05-05 18:53:33,988 | angr.storage.memory_mixins.default_filler_mixin | Filling memory at 0x7ffffffffff0088 with 8 unconstrained bytes referenced from 0xe15ec60 (trace_printf+0x94 in tee.elf (0xe15ec60))
WARNING  | 2024-05-05 18:53:34,092 | angr.storage.memory_mixins.default_filler_mixin | Filling memory at 0xff00000000000000 with 8 unconstrained bytes referenced from 0xe15ec64 (trace_printf+0x98 in tee.elf (0xe15ec64))


KeyboardInterrupt: 

In [17]:
vars(simgr)

{'_project': <Project ../optee/optee_os/out/arm/core/tee.elf>,
 'completion_mode': <function any(iterable, /)>,
 '_errored': [<State errored with "IR decoding error at 0xe1039ac. You can hook this instruction with a python replacement using project.hook(0xe1039ac, your_function, length=length_of_instruction).">],
 '_lock': <angr.misc.picklable_lock.PicklableLock at 0x7f7dcc9640a0>,
 '_stashes': defaultdict(list,
             {'active': [<SimState @ 0xe15ebcc>,
               <SimState @ 0xe15c2ec>,
               <SimState @ 0xe116654>,
               <SimState @ 0xe1038d4>,
               <SimState @ 0xe10398c>,
               <SimState @ 0xe103994>,
               <SimState @ 0xe11a3a0>,
               <SimState @ 0xe1038c8>,
               <SimState @ 0xe104a44>,
               <SimState @ 0xe11a3a0>,
               <SimState @ 0xe1038c8>,
               <SimState @ 0xe1038c0>,
               <SimState @ 0xe15ec5c>,
               <SimState @ 0xe1038c0>,
               <SimState @ 0

In [ ]:
print(flag)